In [1]:
import sqlite3
import datetime as dt

In [ ]:
sqlite3.sqlite_version

In [ ]:
# Connect to an in-memory database.
db = sqlite3.connect(":memory:")
db

In [ ]:
# Create a Cursor object.
cursor = db.cursor()
cursor

In [ ]:
# Attributes from the Cursor.
# - execute methods -> Perform SQL statements.
# - fetch methods   -> Fetch data from a SELECT resultset.
# - description     -> Metadata for a SELECT resultset.
# - rowcount        -> Rows affected by the last SQL statement.
for attribute in dir(cursor):
    if attribute.startswith("_"):
        continue
    print(attribute)

## Create table

In [ ]:
# Create a customer table.
cursor.execute(
    """
    CREATE TABLE Klanten (
        KlantId INTEGER,
        Voornaam TEXT,
        Achternaam TEXT NOT NULL,
        GeboorteDatum DATE,
        PRIMARY KEY(KlantId)
    );
    """
)

## Inserting data

In [ ]:
# Insert two new customers.
cursor.execute(
    """
    INSERT INTO Klanten
        (Voornaam, Achternaam, GeboorteDatum)
    VALUES
        ('Ingrid', 'Jansen', '1984-09-06'),
        ('Henk', 'Knol', '1957-08-23')
    ;
    """
)

In [ ]:
# Verify the insert statement worked...
cursor.rowcount

In [9]:
# Customer data as dict.
customer = {
    "Voornaam": "Maria",
    "Achternaam": "Klomp",
    "GeboorteDatum": dt.date(1990, 5, 7)
}


In [10]:
# Define query as a template...
template = """
    INSERT INTO Klanten
        (Voornaam, Achternaam, GeboorteDatum)
    VALUES (
        '{Voornaam}', '{Achternaam}', '{GeboorteDatum}'
    );
"""

In [ ]:
# Insert values using Python formatting...
print(template.format(**customer))

In [12]:
# Customer data with missing value.
customer = {
    "Voornaam": "Maria",
    "Achternaam": "Klomp",
    "GeboorteDatum": None,
}

In [ ]:
# Bad idea...
print(template.format(**customer))

In [ ]:
# Good idea: Use a parametrized query.
cursor.execute(
    """
    INSERT INTO Klanten
        (Voornaam, Achternaam, GeboorteDatum)
    VALUES (:Voornaam, :Achternaam, :GeboorteDatum);
    """,
    customer
)
cursor.rowcount

In [ ]:
# Using positional parameters.
cursor.execute(
    """
    INSERT INTO Klanten
        (Voornaam, Achternaam, GeboorteDatum)
    VALUES (?, ?, ?);
    """,
    ("Piet", "Klaassen", dt.date(1978, 9, 4))
)
cursor.rowcount

In [16]:
# List of customer records.
customers = [
    {"Voornaam": "Noah", "Achternaam": "Versteeg", "GeboorteDatum": None},
    {"Voornaam": "Mark", "Achternaam": "Vos", "GeboorteDatum": dt.date(1998, 1, 4)},
    {"Voornaam": "Bart", "Achternaam": "Poot", "GeboorteDatum": dt.date(1955, 8, 21)},
]

In [ ]:
cursor.executemany(
    """
    INSERT INTO Klanten
        (Voornaam, Achternaam, GeboorteDatum)
    VALUES (:Voornaam, :Achternaam, :GeboorteDatum);
    """,
    customers
)
cursor.rowcount

## Select data

In [ ]:
# Perform a SELECT query.
cursor.execute("SELECT * FROM Klanten")

In [ ]:
# Look at the metadata
# - name
# - type_code
# - display_size
# - internal_size
# - precision
# - scale
# - null_ok
cursor.description

In [ ]:
# Extract column names from the description.
column_names = [field_meta[0] for field_meta in cursor.description]
column_names

In [ ]:
# Use fetch methods to get the data.
row = cursor.fetchone()
row

In [ ]:
# Row is a simple tuple.
type(row)

In [ ]:
# Convert to dict.
dict(zip(column_names, row))

## Row Factory

In [34]:
# Set a row factory.
db.row_factory = sqlite3.Row

In [35]:
# Create a new cursor using the row factory.
cursor = db.cursor()

In [36]:
# Perform a query and fetch the first row.
cursor.execute("SELECT * FROM Klanten;")
row = cursor.fetchone()

In [ ]:
# Note that result is now a Row object.
type(row)

In [ ]:
# Get the column names from the Row object.
row.keys()

In [ ]:
# Index by column name.
row["Voornaam"]

In [ ]:
# Or by index.
row[1]

In [ ]:
rows = cursor.fetchall()
for row in rows:
    print(dict(row))

In [42]:
# Close the connection.
# Note: Closes automatically when going out of scope.
db.close()